In [2]:
# 必要なパッケージのインストール
!pip install nltk
!pip install gensim
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
import numpy as np
import pandas as pd

In [4]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# 不要なタグや記号などの削除
import re
def cleaning_text(text):
    pattern1 = '-*' # ハイフンの削除
    text = re.sub(pattern1, '', text)
    pattern2 = '`*' # クォーテーションの削除
    text = re.sub(pattern2, '', text)
    pattern2 = "'*" # クォーテーションの削除
    text = re.sub(pattern2, '', text)
    pattern3 = '\(.*?\)' # かっこ内の削除
    text = re.sub(pattern3, '', text)
    pattern4 = '<.*?>' # かっこ内の削除
    text = re.sub(pattern4, '', text)
    return text

In [6]:
# スペースで区切る
def tokenize_text(text):
  text = re.sub('[.,]', '', text)
  return text.split()

In [7]:
def remove_latter(text):
  pattern = '/.*'
  text = re.sub(pattern, '', text)
  return text

In [8]:
#lemmatizeを行う
from nltk.corpus import wordnet as wn

def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma

In [132]:
# ストップワードの削除
en_stop = nltk.corpus.stopwords.words('english')
en_stop= ["``", "/", ";", "--", ":", ")", "(", '"', '&', "'", '-', "?", "!", ">", "<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12"]                                                      \
         +["said","say","u","v","wa","ct","ha","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop
#         +["said","say","u","v","wa","ct","ha","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price","make","would","could","may","one","two","new","know","take","look","like","go","come","get","use","many","first","also","years","af","must","seem","well","us","still","tell","much","even","give","show","become","around","another","see","last","since","good","want","might"]          \

def remove_stopwords(word, stopwordset):
  if word in stopwordset:
    return None
  else:
    return word

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [133]:
# テキストの前処理
def preprocessing_text(text):
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [remove_latter(word) for word in tokens]
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  tokens = [word for word in tokens if word != '']
  tokens = [word for word in tokens if word != '?']
  return tokens

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [134]:
nltk.download("brown")
from nltk.corpus import brown as brown

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [152]:
import numpy
from collections import Counter

def preprocess_documents():
  fileid = brown.fileids()
  preprocessed = [preprocessing_text(brown.raw(fileid[i])) for i in range(len(fileid))]

  # 共通してよく使われる単語を消去する
  all_words = []
  for i in range(len(preprocessed)):
    all_words += preprocessed[i] 
  c = Counter(all_words)
  common = set([x[0] for x in c.most_common(250)])
  for i in range(len(preprocessed)):
    new_list = [x for x in preprocessed[i] if x not in common]
    preprocessed[i] = new_list

  return preprocessed


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [153]:
import gensim
from gensim import corpora

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [154]:
#documentを，gensim LDAが読み込めるデータ構造にする
preprocessed = preprocess_documents()

#辞書の作成
dictionary = corpora.Dictionary(preprocessed)

#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocessed]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [198]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=15,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           random_state=1,
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [199]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.001*"position" + 0.001*"material" + 0.001*"trial" + 0.001*"pressure" + 0.001*"volume" + 0.001*"moment" + 0.001*"department" + 0.001*"cannot" + 0.001*"million" + 0.001*"either"')
(1, '0.001*"death" + 0.001*"atom" + 0.001*"person" + 0.001*"human" + 0.001*"field" + 0.001*"story" + 0.001*"sound" + 0.001*"earth" + 0.001*"history" + 0.001*"costs"')
(2, '0.001*"class" + 0.001*"mean" + 0.001*"walk" + 0.001*"costs" + 0.001*"cost" + 0.001*"across" + 0.001*"history" + 0.001*"believe" + 0.001*"center" + 0.001*"im"')
(3, '0.001*"court" + 0.001*"wife" + 0.001*"moment" + 0.001*"project" + 0.001*"available" + 0.001*"policy" + 0.001*"perhaps" + 0.001*"department" + 0.001*"word" + 0.001*"determine"')
(4, '0.001*"federal" + 0.001*"information" + 0.001*"secretary" + 0.001*"court" + 0.001*"shall" + 0.001*"brown" + 0.001*"cell" + 0.001*"believe" + 0.001*"person" + 0.001*"complete"')
(5, '0.001*"perhaps" + 0.001*"talk" + 0.001*"money" + 0.001*"policy" + 0.001*"tax" + 0.001*"whether" + 0.001*"section" 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [200]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.95389336), (9, 0.044652265)]
document ID 1:[(0, 0.9873477)]
document ID 2:[(0, 0.4750748), (3, 0.044613063), (4, 0.45510563), (9, 0.023899343)]
document ID 3:[(11, 0.9982761)]
document ID 4:[(2, 0.01312704), (4, 0.30770332), (6, 0.08891445), (9, 0.018199274), (11, 0.49051115), (12, 0.06817361)]
document ID 5:[(9, 0.63746166), (11, 0.35382593)]
document ID 6:[(6, 0.09322124), (11, 0.90523326)]
document ID 7:[(2, 0.10245078), (9, 0.35442093), (12, 0.5342743)]
document ID 8:[(3, 0.9088241), (11, 0.08973575)]
document ID 9:[(3, 0.5781798), (4, 0.013383803), (6, 0.021793637), (9, 0.30222195), (10, 0.07823756)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [201]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [202]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [203]:
import random
# 10個の文書のトピック分布を出力
categories=[brown.categories(fileid) for fileid in brown.fileids()]

def print_topic_distribution(n):
  print("document"+str(n)+":")
  print("topic:"+str(ldamodel.get_document_topics(corpus_[n])))
  print("categoriy:"+str(categories[n]))
  print('')

for i in range(10):
  print_topic_distribution(random.randint(0, len(corpus_)-1))

document209:
topic:[(5, 0.83215266), (11, 0.14319436), (12, 0.016357342)]
categoriy:['belles_lettres']

document311:
topic:[(8, 0.9976841)]
categoriy:['learned']

document499:
topic:[(0, 0.044933267), (3, 0.11637401), (7, 0.011560523), (9, 0.07454446), (11, 0.45660663), (12, 0.29010552)]
categoriy:['humor']

document406:
topic:[(2, 0.14863758), (9, 0.039903495), (11, 0.8093032)]
categoriy:['mystery']

document121:
topic:[(4, 0.667861), (14, 0.33050737)]
categoriy:['hobbies']

document278:
topic:[(0, 0.9437074), (4, 0.04538556)]
categoriy:['government']

document20:
topic:[(7, 0.64279526), (9, 0.23376717), (12, 0.1172695)]
categoriy:['news']

document33:
topic:[(0, 0.016928842), (8, 0.57019407), (11, 0.41133365)]
categoriy:['news']

document428:
topic:[(2, 0.7934462), (5, 0.199882)]
categoriy:['science_fiction']

document336:
topic:[(4, 0.9983636)]
categoriy:['learned']



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
